In [15]:
import cv2
import convolution
import numpy as np
import math
import random

In [16]:
image_input_file = 'in.jpg'
image_output_file = 'out.jpg'

capture = None
function = None
unprocessed_image = None
processed_image = None

deriv_length = 3
vector_length = 10
current_channel = -1

smoothing_amount_s = None
smoothing_amount_S = None
angle = None
pixels_interval = None

max_smoothing_amount = 31
max_angle = 360
max_pixels_interval = 100

window_name = 'Processed Image'
handler_name = 'Handler'

In [17]:
def smoothing_s(image, smoothing_amount):
    smooth_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    standard_deviation_x = smoothing_amount / 5
    standard_deviation_y = smoothing_amount / 5
    smooth_image = cv2.GaussianBlur(smooth_image,(smoothing_amount,smoothing_amount),standard_deviation_x,standard_deviation_y)
    return smooth_image

def smoothing_S(image, smoothing_amount):
    smooth_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    standard_deviation_x = smoothing_amount / 5
    standard_deviation_y = smoothing_amount / 5    
    filter_x = cv2.getGaussianKernel(smoothing_amount, standard_deviation_x)
    filter_y = cv2.getGaussianKernel(smoothing_amount, standard_deviation_y)
    filter_xy = np.multiply(filter_y, filter_x.T)
    smooth_image = convolution.convolution_2d(smooth_image, filter_xy)
    return smooth_image

def grad_x(image, deriv_length):
    kx, ky = cv2.getDerivKernels(1, 0, deriv_length, True)

    filter_x = np.multiply(ky, kx.T)
    gradient_x = cv2.filter2D(image, -1, filter_x)
    
    gradient_x = np.float64(gradient_x)
    gradient_x -= np.amin(gradient_x)
    gradient_x *= (255.0 / np.amax(gradient_x))
    gradient_x = np.uint8(gradient_x)
    return gradient_x

def grad_y(image, deriv_length):
    kx, ky = cv2.getDerivKernels(0, 1, deriv_length, True)
    
    filter_y = np.multiply(ky, kx.T)
    gradient_y = cv2.filter2D(image, -1, filter_y)
        
    gradient_y = np.float64(gradient_y)
    gradient_y -= np.amin(gradient_y)
    gradient_y *= (255.0 / np.amax(gradient_y))
    gradient_y = np.uint8(gradient_y)
    return gradient_y

def mag_grad(grad_x, grad_y):
    mag_grad = np.hypot(grad_x,grad_y)
    mag_grad = np.float64(mag_grad)
    mag_grad -= np.amin(mag_grad)
    mag_grad *= (255.0 / np.amax(mag_grad))
    mag_grad = np.uint8(mag_grad)
    return mag_grad

def grad_vector(image, deriv_length, pixels_interval, vector_length):
    grad_vector_image = cv2.cvtColor(unprocessed_image, cv2.COLOR_BGR2GRAY)
    
    gradient_x = grad_x(grad_vector_image, deriv_length)
    gradient_y = grad_y(grad_vector_image, deriv_length)
    mag_gradient = mag_grad(gradient_x, gradient_y)

    for row in range(0, grad_vector_image.shape[0], pixels_interval):
        for col in range(0, grad_vector_image.shape[1], pixels_interval):
            g_x = gradient_x[row][col]
            g_y = gradient_y[row][col]
            m_g = mag_gradient[row][col]
            
            if m_g:
                angle = None
                if g_x:
                    angle = math.atan(g_y / g_x)
                else:
                    angle = 90 * math.pi / 180.0
                if angle:
                    vec_x = m_g * math.cos(angle)
                    vec_y = m_g * math.sin(angle)

                    vec_x *= (vector_length / m_g)
                    vec_y *= (vector_length / m_g)

                    vec_x += row
                    vec_y += col

                    vec_x = int(vec_x)
                    vec_y = int(vec_y)

                    cv2.arrowedLine(grad_vector_image, (col, row), (vec_y, vec_x), (0,255,0), 2)
    return grad_vector_image

def rotating(image, agl):
    rotate_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (h,w) = rotate_image.shape[:2]
    (ch, cw) = (h//2, w//2)
    M = cv2.getRotationMatrix2D((cw, ch), -agl, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nw = int((h * sin) + (w * cos))
    nh = int((h * cos) + (w * sin))
    M[0, 2] += (nw / 2) - cw
    M[1, 2] += (nh / 2) - ch
    rotate_image = cv2.warpAffine(rotate_image, M, (nw, nh))
    return rotate_image

In [18]:
def smoothing_handler_s(smam):
    global smoothing_amount_s
    if smam > 0:
            smoothing_amount_s = smam
            if smoothing_amount_s % 2 == 0:
                smoothing_amount_s += 1
    return

def smoothing_handler_S(smam):
    global smoothing_amount_S
    if smam > 0:
            smoothing_amount_S = smam
            if smoothing_amount_S % 2 == 0:
                smoothing_amount_S += 1
    return

def gradient_vector_handler_p(pxls):
    global pixels_interval
    if pxls > 0:
        pixels_interval = pxls
    return

def angle_handler_r(agl):
    global angle
    if agl >= 0:
        angle = agl
    return

In [19]:
def fun_i():
    global unprocessed_image, processed_image, image_input_file
    unprocessed_image = cv2.imread(image_input_file)
    processed_image = np.copy(unprocessed_image)
    return

def fun_I():
    global unprocessed_image, processed_image, capture
    valid_frame, captured_image = capture.read()
    unprocessed_image = captured_image
    processed_image = np.copy(unprocessed_image)
    return

def fun_w():
    global processed_image, image_output_file
    cv2.imwrite(image_output_file, processed_image)
    return

def fun_g():
    global unprocessed_image, processed_image
    processed_image = cv2.cvtColor(unprocessed_image, cv2.COLOR_BGR2GRAY)
    return

def fun_G():
    global unprocessed_image, processed_image
    r, g, b = unprocessed_image[:,:,2], unprocessed_image[:,:,1], unprocessed_image[:,:,0]
    processed_image = (0.299*r) + (0.587*g) + (0.114*b)
    processed_image = np.uint8(processed_image)
    return

def fun_c():
    global current_channel
    current_channel = (current_channel+1)%3
    return

def fun_s():
    global unprocessed_image, processed_image, smoothing_amount_s
    processed_image = smoothing_s(unprocessed_image, smoothing_amount_s)
    return
        
def fun_S():
    global unprocessed_image, processed_image, smoothing_amount_S
    processed_image = smoothing_S(unprocessed_image, smoothing_amount_S)
    return

def fun_d():
    global unprocessed_image, processed_image
    processed_image = cv2.resize(unprocessed_image, (int(unprocessed_image.shape[1]/2), int(unprocessed_image.shape[0]/2)), interpolation=cv2.INTER_AREA)
    return
    

def fun_D():
    global unprocessed_image, processed_image, max_smoothing_amount
    standard_deviation_x = max_smoothing_amount / 5
    standard_deviation_y = max_smoothing_amount / 5
    processed_image = cv2.GaussianBlur(unprocessed_image,(max_smoothing_amount,max_smoothing_amount),standard_deviation_x,standard_deviation_y)
    processed_image = cv2.resize(processed_image, (int(processed_image.shape[1]/2), int(processed_image.shape[0]/2)), interpolation=cv2.INTER_AREA)
    return

def fun_x():
    global unprocessed_image, processed_image, deriv_length
    processed_image = cv2.cvtColor(unprocessed_image, cv2.COLOR_BGR2GRAY)
    gradient_x = grad_x(processed_image, deriv_length)
    processed_image = gradient_x
    return

def fun_y():
    global unprocessed_image, processed_image, deriv_length
    processed_image = cv2.cvtColor(unprocessed_image, cv2.COLOR_BGR2GRAY)
    gradient_y = grad_y(processed_image, deriv_length)       
    processed_image = gradient_y
    return

def fun_m():
    global unprocessed_image, processed_image, deriv_length
    processed_image = cv2.cvtColor(unprocessed_image, cv2.COLOR_BGR2GRAY)
    gradient_x = grad_x(processed_image, deriv_length)
    gradient_y = grad_y(processed_image, deriv_length)
    mag_gradient = mag_grad(gradient_x, gradient_y)
    processed_image = mag_gradient
    return

def fun_p():
    global unprocessed_image, processed_image, deriv_length, pixels_interval, vector_length
    processed_image = grad_vector(unprocessed_image, deriv_length, pixels_interval, vector_length)
    return
        
def fun_r():
    global unprocessed_image, processed_image, angle
    processed_image = rotating(unprocessed_image, angle)
    return

def fun_h():
    print('This is a program to perfomr simple image manipulation useing openCV. The program loads an image from a file. When user presses a key perfomr the operation corresponding to the key on the original image (not the result of the last processing step.)')
    print('The program is satisfing the following specifications:')
    print('1] The image should be either read from a file or captured directly from a camera. When capturing an image from the camera, continue to capture and process image continuously.')
    print('2] The image is read as 3 channel color image.')
    print('3] The program works for any size image.')
    print('\n')
    print('Supported keys are as below:\n')
    print('i -> Reload the image (i.e. cancel any previous processing.)')
    print('I -> Capture image from camera.')
    print('w -> Save the current (possibly processed) image into the file \'out.jpg\'.')
    print('g -> Convert the image to grayscale using openCV.')
    print('G -> Convert the image to grayscale using implementation.')
    print('c -> Cycle through the color (RGB) channels of the image showing a different channel every time the key is pressed.')
    print('s -> Convert the image to grayscale and smooth it using openCV function. A trackbar is provided to control the amount of smoothing.')
    print('S -> Convert the image to grayscale and smooth it using an implementation of convolution. A trackbar is provided to control the amount of smoothing.')
    print('d -> Downsample the image by a factor of 2 without smoothing.')
    print('D -> Downsample the image by a factor of 2 with smoothing.')
    print('x -> Convert to grayscale and perform convolution with an x derivative filter. Normalize to the range [0,255].')
    print('y -> Convert to grayscale and perform convolution with an y derivative filter. Normalize to the range [0,255].')
    print('m -> Show the magnitude of the gradient normalized to the range [0,255].')
    print('p -> Convert the image to grayscale and plot gradient vectors of the image for every N pixels and have a lenght of K. A trackbar is provided to control N.')
    print('r -> Convert the image to grayscale and rotate it using an angle of theta degrees. A trackbar is provided to control the rotation angle theta.')
    print('h -> Display help.')
    print('e or <ESC> -> Exit from an application.')
    return

In [20]:
def init_app():
    global function, smoothing_amount_s, smoothing_amount_S, angle, current_channel, pixels_interval
    function = None
    smoothing_amount_s = None
    smoothing_amount_S = None
    angle = None
    current_channel = -1
    pixels_interval = None
    cv2.destroyAllWindows()
    return

def init_app_c():
    global function, smoothing_amount_s, smoothing_amount_S, angle, pixels_interval
    function = None
    smoothing_amount_s = None
    smoothing_amount_S = None
    angle = None
    pixels_interval = None
    cv2.destroyAllWindows()
    return

In [21]:
while(True):
    
    cv2.namedWindow(window_name)
    
    key = cv2.waitKey(100) & 0xFF
    
    if key == 27 or key == ord('e'):
        init_app()
        if capture is not None:
            capture.release() 
        cv2.destroyAllWindows()
        break
        
    if key == ord('i'):
        init_app()
        capture = None
    
    if key == ord('I'):
        init_app()
        capture = cv2.VideoCapture(0)
        
    if key == ord('h'):
        fun_h()
    
    if key == ord('w'):
        fun_w()
        
    if key == ord('g'):
        init_app()
        function = fun_g
    
    if key == ord('G'):
        init_app()
        function = fun_G
    
    if key == ord('c'):
        init_app_c()
        fun_c()
    
    if key == ord('s'):
        init_app()
        smoothing_amount_s = 1
        function = fun_s
    
    if key == ord('S'):
        init_app()
        smoothing_amount_S = 1
        function = fun_S
        
    if key == ord('d'):
        init_app()
        function = fun_d
        
    if key == ord('D'):
        init_app()
        function = fun_D
        
    if key == ord('x'):
        init_app()
        function = fun_x
    
    if key == ord('y'):
        init_app()
        function = fun_y
    
    if key == ord('m'):
        init_app()
        function = fun_m
    
    if key == ord('p'):
        init_app()
        pixels_interval = max_pixels_interval
        function = fun_p
    
    if key == ord('r'):
        init_app()
        angle = 0
        function = fun_r
    
    if capture is None:
        fun_i()
    else:
        fun_I()
    
    if smoothing_amount_s is not None:
        cv2.createTrackbar(handler_name, window_name, smoothing_amount_s, max_smoothing_amount, smoothing_handler_s)

    if smoothing_amount_S is not None:
        cv2.createTrackbar(handler_name, window_name, smoothing_amount_S, max_smoothing_amount, smoothing_handler_S)

    if angle is not None:
        cv2.createTrackbar(handler_name, window_name, angle, max_angle, angle_handler_r)

    if pixels_interval is not None:
        cv2.createTrackbar(handler_name, window_name, pixels_interval, max_pixels_interval, gradient_vector_handler_p)
    
    if function is not None:
        function()
    
    if current_channel > -1:
        processed_image = unprocessed_image[:,:,current_channel]
    
    cv2.imshow(window_name,processed_image)